In [12]:
import numpy as np
import tensorflow as tf

Y=np.load('Test_dis.npy')
node=np.load('Test_Node.npy')
para=np.load('beam_test.npy')

In [11]:
beam_exp_np = np.repeat(
    para[:, np.newaxis, :],      # 从 (500,2) -> (500,1,2)
    repeats=node.shape[1],       # 在第二个轴(231)上重复
    axis=1
)

In [13]:
X = np.concatenate([node, beam_exp_np], axis=-1)
print("X.shape:", X.shape)

X.shape: (500, 231, 4)


In [14]:
Y=Y.reshape(Y.shape[0], Y.shape[1], -1)
# Y=Y[:,20,:]
print("Y.shape:", Y.shape)  # (500, 231, 20)

Y.shape: (500, 231, 20)


In [15]:
import pickle

pickle.dump(X, open("X_beam.pkl", "wb"))
pickle.dump(Y, open("Y_beam_2.pkl", "wb"))

In [17]:
from itertools import combinations
def generate_pairs(arr):
    # Shape of the input array
    batch_size, num_elements, pair_length = arr.shape

    # We know we need combinations of 2 from the last dimension
    pairs = []
    for i in range(batch_size):
        for j in range(num_elements):
            elements = arr[i, j]
            # Generate all unique pairs
            pair_list = [list(pair) for pair in combinations(elements, 2)]
            pairs.append(pair_list)

    # Convert the list of pairs to a numpy array
    pairs_array = np.array(pairs)

    # Reshape to the desired output format
    # Combine second and third dimensions
    pairs_array = pairs_array.reshape(batch_size, -1, 2)

    return pairs_array

def unique_pairs(pairs_array):
    unique_pairs_list = []
    
    for batch in pairs_array:
        # Use np.unique to remove duplicates within each batch
        # In this case, we are not sorting the pairs
        unique_pairs = np.unique(batch, axis=0)
        unique_pairs_list.append(unique_pairs)
    
    # Find the maximum number of unique pairs to pad the batches to the same length
    max_length = max(len(batch) for batch in unique_pairs_list)
    
    # Pad all batches to the same length
    padded_batches = []
    for batch in unique_pairs_list:
        pad_width = max_length - len(batch)
        padded_batch = np.pad(batch, ((0, pad_width), (0, 0)), mode='constant')
        padded_batches.append(padded_batch)
    
    # Convert to a numpy array
    result = np.array(padded_batches)
    
    return result

# Example usage
input_array =np.load('Test_edge.npy')
output_array = generate_pairs(input_array)
unique_output_array = unique_pairs(output_array)

print("Shape before uniqueness check:", output_array.shape)  # Shape before uniqueness check
print("Shape after uniqueness check:", unique_output_array.shape)   # Shape after uniqueness check

print(unique_output_array[0])

Shape before uniqueness check: (500, 1200, 2)
Shape after uniqueness check: (500, 630, 2)
[[  0   1]
 [  0  21]
 [  0  22]
 ...
 [227 228]
 [228 229]
 [229 230]]


In [18]:
new_array = np.ones((500, 630, 1))  # For example, fill it with ones

# Concatenate along the third dimension
combined_array = np.concatenate((unique_output_array, new_array), axis=2)

In [19]:

# Define the filename to save the array
filename = "new_edge.npy"

# Save the array to a .npy file
np.save(filename, combined_array)